In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import warnings

from keras.layers import Input, Embedding, Flatten, Dot, Dense
from keras.models import Model

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras import Sequential

warnings.filterwarnings('ignore')
%matplotlib inline

Using TensorFlow backend.


In [2]:
def combine_columns_to_embed(df):
    df['docs_to_embed'] = ''

    for idx, row in df.iterrows():
        new_str = '{}, {}, {}'.format(row['product_title'], row['review_headline'], row['review_body'])
        df.loc[idx, 'docs_to_embed'] = new_str
    
    return df



def read_data(f_name):
    data = []
    count = 0
    with open(f_name) as f:
        for row in f.readlines():
            data.append(row.strip('\n').split('\t'))
#             count += 1
#             print (count)
#             if count > 100:
#                 print ('tripped the if statement!')
#                 break
                
    df = pd.DataFrame(data[1:], columns = data[0])
    
    df.drop(['marketplace', 'review_id', 'product_parent'], axis=1, inplace=True)
    df = combine_columns_to_embed(df)

    
    return df

# need some help on figuring out why this takes forever to run
# df = read_data('amazon_reviews_us_Books_v1_00.tsv')


df = read_data('sample_us.tsv')

In [3]:
df.shape

(49, 13)

In [4]:
df.dtypes

customer_id          object
product_id           object
product_title        object
product_category     object
star_rating          object
helpful_votes        object
total_votes          object
vine                 object
verified_purchase    object
review_headline      object
review_body          object
review_date          object
docs_to_embed        object
dtype: object

In [5]:
cols_for_embeddings = ['product_title', 'review_headline', 'review_body']

In [6]:
cols_without_embeddings = ['customer_id', 'product_id', 'product_category', 'votes', 'vine', 'verified_purchase', 'date']

In [ ]:
# maybe consider date a running variable? 

In [7]:
def get_label(df):
    labels = [1 if int(x) >= 4 else 0 for x in df['star_rating'] ]
    return labels

labels = get_label(df)

## Build and train a model

### Features to embed:
1. Product title
2. Review headline
3. Review body
4. Customer ID
5. Product ID 

In [8]:
def get_encoded_ids(df, id_name):

    assert id_name in ['customer_id', 'product_id', 'docs_to_embed']
    
    vocab_size = get_vocab_size(df, col_name = id_name)

    docs = df[id_name].values.tolist()
    
    encoded_ids = [one_hot(d, vocab_size) for d in docs]

    return np.array(encoded_ids)

def get_vocab_size(df, col_name):
    vocab_size = len(set((' ').join(df[col_name]).split()))
    return vocab_size

def get_max_length(df, col_name):
    max_length = 0
    for idx, row in df.iterrows():
        doc = row[col_name]
        l = len(doc.split())
        if l > max_length:
            max_length = l
            
    return max_length

def get_padded_documents(df):
    encoded_docs = get_encoded_ids(df, 'docs_to_embed')
    max_length = get_max_length(df, 'docs_to_embed')
    
    padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
    return padded_docs

### Add additional inputs
On top of looking at the textual data, we have a few additional columns we need to consider. Each of these we'll consider a running variable
1. Date
2. Product category
3. Votes

We'll specify all of these as  inputs, then pass them in as a list to our model

In [9]:
def get_model_input_specs(df):
    vocab_size = get_vocab_size(df, 'docs_to_embed')
    max_length = get_max_length(df, col_name = 'docs_to_embed')

    n_users = len(set(df['customer_id'].values.tolist()))
    n_products = len(set(df['product_id'].values.tolist()))

    return vocab_size, max_length, n_users, n_products
    


In [10]:
def get_model():
    
    vocab_size, max_length, n_users, n_products = get_model_input_specs(df)
    

    doc_input = Input(shape=[max_length,], name="Document-Input")
    doc_embedding = Embedding(vocab_size, 8, name="Document-Embedding", input_length=max_length)(doc_input)
    doc_vec = Flatten(name="Flatten-Documents")(doc_embedding)


    product_input = Input(shape=[1], name="Product-Input")
    product_embedding = Embedding(n_products+1, 5, name="Product-Embedding")(product_input)
    product_vec = Flatten(name="Flatten-Products")(product_embedding)

    user_input = Input(shape=[1], name="User-Input")
    user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
    # user_embedding = Embedding(n_users, 5, name="User-Embedding")(user_input)

    user_vec = Flatten(name="Flatten-Users")(user_embedding)


    prod = Dot(name="Dot-Product", axes=1)([product_vec, user_vec])

    # your model is a list of embedded inputs, then the dot product
    model = Model([user_input, product_input, doc_input], prod)

    model.compile('adam', 'mean_squared_error')
    
    return model

In [11]:
def get_model_input_data(df):
    
    # get padded docs here 
    padded_docs = get_padded_documents(df)
    
    encoded_product_ids = get_encoded_ids(df, 'product_id')
    encoded_customer_idx = get_encoded_ids(df, 'customer_id')

    return [encoded_customer_idx, encoded_product_ids, padded_docs]

### Train the model!

In [12]:
model = get_model()

data_input = get_model_input_data(df)

history = model.fit(data_input, df.star_rating, epochs=500, verbose=1)


model.save('regression_model.h5')

W0819 17:35:55.494217 140279529641792 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0819 17:35:55.512439 140279529641792 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0819 17:35:55.554687 140279529641792 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0819 17:35:55.555496 140279529641792 deprecation_wrapper.py:119] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.t

Epoch 1/500
49/49 [==============================] - 1s 11ms/step - loss: 19.3228
Epoch 2/500
49/49 [==============================] - 0s 46us/step - loss: 19.3202
Epoch 3/500
49/49 [==============================] - 0s 43us/step - loss: 19.3178
Epoch 4/500
49/49 [==============================] - 0s 44us/step - loss: 19.3152
Epoch 5/500
49/49 [==============================] - 0s 44us/step - loss: 19.3125
Epoch 6/500
49/49 [==============================] - 0s 42us/step - loss: 19.3098
Epoch 7/500
49/49 [==============================] - 0s 46us/step - loss: 19.3071
Epoch 8/500
49/49 [==============================] - 0s 47us/step - loss: 19.3043
Epoch 9/500
49/49 [==============================] - 0s 44us/step - loss: 19.3014
Epoch 10/500
49/49 [==============================] - 0s 45us/step - loss: 19.2984
Epoch 11/500
49/49 [==============================] - 0s 47us/step - loss: 19.2954
Epoch 12/500
49/49 [==============================] - 0s 44us/step - loss: 19.2921
Epoch 13/500


49/49 [==============================] - 0s 42us/step - loss: 17.6841
Epoch 100/500
49/49 [==============================] - 0s 46us/step - loss: 17.6485
Epoch 101/500
49/49 [==============================] - 0s 43us/step - loss: 17.6102
Epoch 102/500
49/49 [==============================] - 0s 43us/step - loss: 17.5714
Epoch 103/500
49/49 [==============================] - 0s 42us/step - loss: 17.5334
Epoch 104/500
49/49 [==============================] - 0s 42us/step - loss: 17.4940
Epoch 105/500
49/49 [==============================] - 0s 46us/step - loss: 17.4545
Epoch 106/500
49/49 [==============================] - 0s 43us/step - loss: 17.4155
Epoch 107/500
49/49 [==============================] - 0s 41us/step - loss: 17.3749
Epoch 108/500
49/49 [==============================] - 0s 42us/step - loss: 17.3339
Epoch 109/500
49/49 [==============================] - 0s 45us/step - loss: 17.2944
Epoch 110/500
49/49 [==============================] - 0s 40us/step - loss: 17.2525
Epoch 

49/49 [==============================] - 0s 45us/step - loss: 12.6418
Epoch 197/500
49/49 [==============================] - 0s 50us/step - loss: 12.5809
Epoch 198/500
49/49 [==============================] - 0s 41us/step - loss: 12.5198
Epoch 199/500
49/49 [==============================] - 0s 43us/step - loss: 12.4597
Epoch 200/500
49/49 [==============================] - 0s 43us/step - loss: 12.3995
Epoch 201/500
49/49 [==============================] - 0s 45us/step - loss: 12.3364
Epoch 202/500
49/49 [==============================] - 0s 45us/step - loss: 12.2770
Epoch 203/500
49/49 [==============================] - 0s 40us/step - loss: 12.2155
Epoch 204/500
49/49 [==============================] - 0s 42us/step - loss: 12.1555
Epoch 205/500
49/49 [==============================] - 0s 45us/step - loss: 12.0935
Epoch 206/500
49/49 [==============================] - 0s 42us/step - loss: 12.0333
Epoch 207/500
49/49 [==============================] - 0s 41us/step - loss: 11.9710
Epoch 

49/49 [==============================] - 0s 45us/step - loss: 6.8700
Epoch 295/500
49/49 [==============================] - 0s 41us/step - loss: 6.8170
Epoch 296/500
49/49 [==============================] - 0s 44us/step - loss: 6.7653
Epoch 297/500
49/49 [==============================] - 0s 42us/step - loss: 6.7137
Epoch 298/500
49/49 [==============================] - 0s 41us/step - loss: 6.6613
Epoch 299/500
49/49 [==============================] - 0s 51us/step - loss: 6.6098
Epoch 300/500
49/49 [==============================] - 0s 50us/step - loss: 6.5573
Epoch 301/500
49/49 [==============================] - 0s 44us/step - loss: 6.5066
Epoch 302/500
49/49 [==============================] - 0s 63us/step - loss: 6.4542
Epoch 303/500
49/49 [==============================] - 0s 50us/step - loss: 6.4053
Epoch 304/500
49/49 [==============================] - 0s 46us/step - loss: 6.3515
Epoch 305/500
49/49 [==============================] - 0s 54us/step - loss: 6.3017
Epoch 306/500
49/4

49/49 [==============================] - 0s 45us/step - loss: 2.8405
Epoch 393/500
49/49 [==============================] - 0s 52us/step - loss: 2.8112
Epoch 394/500
49/49 [==============================] - 0s 43us/step - loss: 2.7834
Epoch 395/500
49/49 [==============================] - 0s 52us/step - loss: 2.7544
Epoch 396/500
49/49 [==============================] - 0s 43us/step - loss: 2.7268
Epoch 397/500
49/49 [==============================] - 0s 45us/step - loss: 2.6981
Epoch 398/500
49/49 [==============================] - 0s 44us/step - loss: 2.6714
Epoch 399/500
49/49 [==============================] - 0s 45us/step - loss: 2.6425
Epoch 400/500
49/49 [==============================] - 0s 44us/step - loss: 2.6158
Epoch 401/500
49/49 [==============================] - 0s 44us/step - loss: 2.5887
Epoch 402/500
49/49 [==============================] - 0s 43us/step - loss: 2.5613
Epoch 403/500
49/49 [==============================] - 0s 46us/step - loss: 2.5351
Epoch 404/500
49/4

49/49 [==============================] - 0s 46us/step - loss: 0.9717
Epoch 491/500
49/49 [==============================] - 0s 47us/step - loss: 0.9607
Epoch 492/500
49/49 [==============================] - 0s 43us/step - loss: 0.9495
Epoch 493/500
49/49 [==============================] - 0s 49us/step - loss: 0.9392
Epoch 494/500
49/49 [==============================] - 0s 44us/step - loss: 0.9291
Epoch 495/500
49/49 [==============================] - 0s 48us/step - loss: 0.9182
Epoch 496/500
49/49 [==============================] - 0s 49us/step - loss: 0.9080
Epoch 497/500
49/49 [==============================] - 0s 48us/step - loss: 0.8979
Epoch 498/500
49/49 [==============================] - 0s 42us/step - loss: 0.8880
Epoch 499/500
49/49 [==============================] - 0s 48us/step - loss: 0.8782
Epoch 500/500
49/49 [==============================] - 0s 43us/step - loss: 0.8682
